# 9.2 Geometric Transformations

- 9.2.1 Affine Transform
- 9.2.2 Perspective Transform

In [ ]:
import cv2

import numpy as np

- <font color="forestgreen">Affine Transformation</font> : 
    - <font color="orange">Combination</font> of several geometric transformations like Translation, Rotation, Scaling, and Shearing.<br><br>
    <img src="resource/affine_transformations.gif" style="height:300px"></img>
    <img src="resource/affine_transformations_2.gif" style="height:300px"></img><br><br>
- <font color="forestgreen">Perspective Transformation</font> : 
    - <font color="orange">Mapping</font> of points from one plane to another plane using <font color="orange">homography matrix</font>.
    - Used to <font color="orange">correct perspective distortion</font> in images.<br><br>
    <img src="resource/proj_rots.gif" style="height:300px"></img>
    <img src="resource/projectives_2.gif" style="height:300px"></img><br><br>

- Ilustration of difference between Affine Transform and Perspective Transform : <br><br>
<img src="resource/TransformationsDifference.png" style="width:500px"></img>

<br><br><br>
### 9.2.1 <font color="orange">Affine Transform</font>
- Affine Transform is a type of geometric transformation that <font color="orange">preserves points, straight lines, and planes.</font>
- In Affine Transform, <font color="orange">parallel lines remain parallel after the transformation.</font>
- Common types of Affine Transformations include Translation, Rotation, Scaling, and Shearing.<br><br>
- To perform Affine Transform, we need to define a <font color="orange">transformation matrix</font> `M`, a 2x3 matrix.
- Transformation matrix `M` can be obtained using the function `cv2.getAffineTransform(pts1, pts2)`.
- Where :
    - `pts1` :  triangle vertices source image
    - `pts2` :  triangle vertices destination image <br><br>
- The transformation is applied using the function `cv2.warpAffine(src, M, dsize)`.
- Where :
    - `src` : source image
    - `M` : transformation matrix
    - `dsize` : size of output image <br><br>
- Illustration of Affine Transform using 3 points mapping : <br><br>
<img src="resource/Warp_Affine_Tutorial_Theory_0.jpg" style="width:300px"></img><br><br>
- After affine transformation, <font color="orange">the shape of the object may change</font>, but the <font color="orange">lines remain straight and parallelism is preserved</font>. <br><br>

<br><br>
## EXAMPLE 1 : Understanding Affine Transform
- Choosing triangle vertices in source and destination images to see how the transformation works.<br><br>

In [ ]:
# load image `lena_border.jpg`
img = cv2.imread('lena_border.jpg')

# get image dimensions
# height, width, channels
h, w, c = img.shape


# define triangle vertices in the input image
# top-left, top-right, bottom-left
pts1 = np.float32([[0, 0], [w-1, 0], [0, h-1]])

# define triangle vertices in the output image with a `horizontal shearing effect`
# top-left, top-right, bottom-left
pts2 = np.float32([[0, 0], [w-1, 0], [int(0.2*w), h-1]])

# get transformation matrix M
M = cv2.getAffineTransform(pts1, pts2)

# apply warp affine
result = cv2.warpAffine(img, M, (w, h))


# display results
cv2.imshow('img', img)
cv2.imshow('result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Above example show how affine transform producing <font color="orange">horizontal shearing effect</font> for destination triangle vertices defined as :
    - top-left vertex : `(0, 0)`
    - top-right vertex : `(w-1, 0)`
    - bottom-left vertex : `(int(0.2*w), h-1)`
- where `w` and `h` are width and height of the source image respectively.
- `0.2*w` indicates that the <font color="orange">bottom-left vertex is shifted 20% of the image width to the right</font>, creating a shearing effect.<br><br><br>
- We can experiment with different combination of triangle vertices to see how the affine transformation affects the image.
    - Try creating <font color="orange">vertical shearing effect</font> by adjusting the y-coordinates of the top-right corner shifted by 20% of the image height to the top ( `0.2*h` ).<br><br>
- Also, try to <font color="orange">vertical and horizontal shearing effect at the same time</font> by adjusting both x and y coordinates of the bottom-left and top-right corners respectively.

<br><br><br><br><br>
### EXAMPLE 2 : Fixing Image Shearing Using Affine Transform
- `book_sheared.jpg` : an image that has been sheared in y direction.
- Goal : to correct the shearing effect using Affine Transform.

<br><br><br><br>

- First, detect the contour of the object in the image to find the bounding box of the object.

In [ ]:
# read image `book_standing.jpg`
img = cv2.imread('book_standing.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# convert to binary image using Otsu thresholding
__, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :

    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # draw bounding rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

# display image
cv2.imshow('img', img)
cv2.waitKey(0)  
cv2.destroyAllWindows()

<br><br><br><br>

- Then, find approximate minimum area rectangle to get the angle of the object using contour approximation.

In [ ]:
# read image `book_standing.jpg`
img = cv2.imread('book_standing.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# convert to binary image using Otsu thresholding
__, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


for cnt in contours :

    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # find minimum enclosing object in clockwise direction 
    perimeter = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)

    # iterate through each corner point and draw circle at each corner point
    for point in approx : 
        cv2.circle(img, point[0], 10, (0, 0, 255), -1)
        cv2.putText(img, f'{point[0][0]}, {point[0][1]}', point[0], cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 1)
        print(point[0])


# display image
cv2.imshow('img', img)
cv2.waitKey(0)  
cv2.destroyAllWindows()

<br><br><br><br>
- Above code show how to find approximated corner points of book object in `book_sheared.jpg` image.<br><br>
<img src="resource/approx-book.png" style="width:300px"></img><br><br>
- We can identify ,
    - 1st corner (top-right) -> `approx[0]`
    - 2nd corner (top-left) -> `approx[1]`
    - 3rd corner (bottom-left) -> `approx[2]`    
    - 4th corner (bottom-right) -> `approx[3]`<br><br>
- We will choose 3 corners from approximated rectangle on book object as triangle vertices in source image for affine transformation.
- We can choose, <font color="orange">top-left</font>, <font color="orange">top-right</font>, and <font color="orange">bottom-left</font> corners as triangle vertices in source image.<br><br>
<img src="resource/Warp_Affine_Tutorial_Theory_1.jpg" style="width:200px"></img><br><br>
- So `pts1` can be defined as :
    - `pts1 = np.array([approx[1], approx[0], approx[2]], np.float32)`<br><br><br><br>

- And finaly we can apply affine transformation to correct the shearing effect by defining `pts2` as the desired triangle vertices in destination image.
- We can define `pts2` as : 
    - `pts2 = np.array([[0, 0], [w,0], [0, h]], np.float32)`
- where `w` and `h` are width and height of the source image respectively.
- The resulting image will have the <font color="orange">shearing effect corrected</font>, making the book appear more rectangular and aligned properly.

In [ ]:
# read image `book_standing.jpg`
img = cv2.imread('book_standing.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# convert to binary image using Otsu thresholding
__, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :

    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)


    # find minimum enclosing object in clockwise direction 
    perimeter = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
    approx = approx[:,0,:] # extract points from array shape (4,1,2) to (4,2)
                                                    

    # construct pts1 which is the triangle vertices source image
    # pts1 (top-left, top-right, bottom-left)
    pts1 = np.array([approx[1], approx[0], approx[2]], np.float32)

    # construct pts2 which is the triangle vertices destination image
    # pts2 (top-left, top-right, bottom-left)
    pts2 = np.array([[0, 0], [w,0], [0, h]], np.float32)

    # get transformation matrix M
    M = cv2.getAffineTransform(pts1, pts2)

    # apply warp affine
    result = cv2.warpAffine(img, M, (w, h))


# display results
cv2.imshow('result', result)
cv2.imshow('img', img)
cv2.imshow('thresh', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

<br><br><br><br><br><br>

____
## 9.2.2 <font color="orange">Perspective Transformation</font>

- M is 3x3 transformation matrix for Perspective Transform.
- to create M matrix, we need **4 points on the input image** and corresponding points on the **output image**
- method `cv2.getPerspectiveTransform(pts1, pts2)`
- where :
    - `pts1` : four vertices source image
    - `pts2` : four vertices destination image <br><br>
<img src="resource/perspective.png" style="width:400px"></img> <br><br><br><br>
### EXAMPLE 3 : Fixing Perspective Distortion in Sudoku Image : <br>
<img src="resource/sudoku.png" style="width:400px"></img> 

- Start with detecting the largest contour in the image to find the Sudoku grid.

In [ ]:
# load image 'sudoku.jpg'
img = cv2.imread('sudoku.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# detect edges using Canny edge detector
edges_img = cv2.Canny(gray, 50, 150, gray)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :
    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # filter out small rects
    if w < 50 or h < 50 :
        continue

    # draw bounding rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

# display image
cv2.imshow('img', img)
cv2.imshow('edges', edges_img)
cv2.waitKey(0)  
cv2.destroyAllWindows()

- Then, find approximate polygon to get the corner points of the Sudoku grid using <font color=orange>contour approximation.</font>

In [ ]:
# load image 'sudoku.jpg'
img = cv2.imread('sudoku.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# detect edges using Canny edge detector
edges_img = cv2.Canny(gray, 50, 150, gray)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :
    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # filter out small rects
    if w < 50 or h < 50 :
        continue
    
    # find minimum enclosing object in clockwise direction 
    perimeter = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
    
    # iterate through each corner point and draw circle at each corner point
    for point in approx : 
        cv2.circle(img, point[0], 10, (0, 0, 255), -1)
        cv2.putText(img, f'{point[0][0]}, {point[0][1]}', point[0], cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 1)
        print(point[0])


# display image
cv2.imshow('img', img)
cv2.imshow('edges', edges_img)
cv2.waitKey(0)  
cv2.destroyAllWindows()

<br><br><br><br>
- Above code show how to find approximated corner points of sudoku grid object in `sudoku.jpg` image.<br><br>
<img src="resource/approx-sudoku.png" style="width:700px"></img><br><br>
- We can identify ,
    - 1st corner (top-right) -> `approx[0]`
    - 2nd corner (top-left) -> `approx[1]`
    - 3rd corner (bottom-left) -> `approx[2]`    
    - 4th corner (bottom-right) -> `approx[3]`<br><br>
- We will re-arrange point position from approximated rectangle on sudoku grid object as : 
    - top-left -> `approx[1]`
    - bottom-left -> `approx[2]`
    - bottom-right -> `approx[3]`
    - top-right -> `approx[0]`
    <br><br>
- So `pts1` can be defined as :
    - `pts1 = np.array([approx[1], approx[2], approx[3], approx[0]], np.float32)`<br><br><br><br>
- And finaly we can apply perspective transformation to correct the perspective distortion by defining `pts2` as the desired corner points in destination image.
- We can define `pts2` as :
    - `pts2 = np.array([[0, 0], [0,h], [w, h], [w, 0]], np.float32)`
    <br><br><br><br> 

In [ ]:
# load image 'sudoku.jpg'
img = cv2.imread('sudoku.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# detect edges using Canny edge detector
edges_img = cv2.Canny(gray, 50, 150, gray)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :
    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # filter out small rects
    if w < 50 or h < 50 :
        continue
    
    # find minimum enclosing object in clockwise direction 
    perimeter = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
    approx = approx[:,0,:] # extract points from array shape (4,1,2) to (4,2)

    
    # define four vertices of source & destination image
    # [top-left, bottom-right, bottom-left, top-right]
    pts1 = np.array([approx[1], approx[2], approx[3], approx[0]], np.float32) # the corder of sudoku image
    pts2 = np.float32([[0, 0], [0,h], [w, h], [w, 0]]) # target image corners


    # create Perpective Transform Matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)

    # Apply Warp perspective transform
    output = cv2.warpPerspective(img, M, (w,h))



# display result
cv2.imshow("Original Image", img)
cv2.imshow("Perspective Transform Image", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

<br><br><br><br>

### EXAMPLE 4 : Correcting Perspective of License Plate Image : <br>

- Start with detecting the largest contour in the image to find the License Plate grid.

In [ ]:
# load image 'license_plate.jpg'
img = cv2.imread('license_plate.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# detect edges using Canny edge detector
edges_img = cv2.Canny(gray, 50, 150, gray)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :
    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # filter out small rects
    if w < 50 or h < 50 :
        continue
    
    # draw bounding rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

# display image
cv2.imshow('img', img)
cv2.imshow('edges', edges_img)
cv2.waitKey(0)  
cv2.destroyAllWindows()

- Then, find approximate polygon to get the corner points of the Sudoku grid using <font color=orange>contour approximation.</font>

In [ ]:
# load image 'license_plate.jpg'
img = cv2.imread('license_plate.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# detect edges using Canny edge detector
edges_img = cv2.Canny(gray, 50, 150, gray)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :
    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # filter out small rects
    if w < 50 or h < 50 :
        continue
    
    # find minimum enclosing object in clockwise direction 
    perimeter = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
    
    # iterate through each corner point and draw circle at each corner point
    for point in approx : 
        cv2.circle(img, point[0], 10, (0, 0, 255), -1)
        cv2.putText(img, f'{point[0][0]}, {point[0][1]}', point[0], cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 1)
        cv2.drawContours(img, [approx], -1, (0,255,0), 2)
        print(point[0])


# display image
cv2.imshow('img', img)
cv2.imshow('edges', edges_img)
cv2.waitKey(0)  
cv2.destroyAllWindows()

<br><br><br><br>
#### ℹ️ <font color=cyan>Alternative Method to Find Corner Points of Object for Rounded Corner - Minimum Area Rectangle</font>
- Above code show how to find approximated corner points of license plate object in `license_plate.jpg` image.<br><br>
<img src="resource/approx-license-plate.png" style="width:500px"></img><br><br>


- ⚠️⚠️⚠️ But, the <font color=orange>approximated point is not correcltly</font> at the corner of license plate object.
- It's caused by the contour approximation is just <font color=orange>simplifies the contour based on curvature</font>.
- When <font color=orange>edge of the object</font> is <font color=orange>not sharp enough</font> (rounded or beveled), the approximated point may not be exactly at the corner.
<br><br><br><br>

- To handle this case, we can use <font color=orange>minimum area rectangle</font> `cv2.minAreaRect()` to find the corner points of the object.
- And use `cv2.boxPoints()` to get the corner points of the rectangle.
- This method will give us more accurate corner points for objects with rounded edges.

In [ ]:
# load image 'license_plate.jpg'
img = cv2.imread('license_plate.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# detect edges using Canny edge detector
edges_img = cv2.Canny(gray, 50, 150, gray)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :
    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # filter out small rects
    if w < 50 or h < 50 :
        continue
    
    # find minimum enclosing object in clockwise direction
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = box.astype(int)

    for point in box : 
        cv2.circle(img, point, 10, (0, 0, 255), -1)
        cv2.putText(img, f'{point[0]}, {point[1]}', point, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 1)
        cv2.drawContours(img, [box], -1, (0,255,0), 2)
        print(point)


# display image
cv2.imshow('img', img)
cv2.imshow('edges', edges_img)
cv2.waitKey(0)  
cv2.destroyAllWindows()

<br><br><br><br>
- Now we have the correct corner points of the license plate object.<br><br>
<img src="resource/min-area-rect-license-plate.png" style="width:500px"></img><br><br>
- We observe that the approximated corner points of license plate object are detected correctly.
    - But the <font color=orange>point order is different</font> from what we found in sudoku grid example.
    - 1st corner (bottom-left) -> `box[0]`
    - 2nd corner (top-left) -> `box[1]`
    - 3rd corner (top-right) -> `box[2]`    
    - 4th corner (bottom-right) -> `box[3]`<br><br>
- This situation can happen depending on the orientation of the object in the image.
    - ⚠️⚠️⚠️ <font color=cyan>OpenCV will not guarantee the order of the points</font> returned by `cv2.minAreaRect()` or even contour approximation. 
    - So we <font color=orange>need to rearrange the points correctly</font> before applying perspective transformation.<br><br><br><br>

- Now re-arrange the point into [top-left, bottom-left, bottom-right, top-right] define `pts1` as :
    - `pts1 = np.array([box[1], box[0], box[3], box[2]], np.float32)`<br><br>
- And finaly we can apply perspective transformation to correct the perspective distortion by defining `pts2` as the desired corner points in destination image.
- We can define `pts2` as :
    - `pts2 = np.array([[0, 0], [0,h], [w, h], [w, 0]], np.float32)`
<br><br><br><br>

In [ ]:
# load image 'license_plate.jpg'
img = cv2.imread('license_plate.jpg')

# get image shape
# h : height, w : width, c : channel
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# detect edges using Canny edge detector
edges_img = cv2.Canny(gray, 50, 150, gray)

# find contour, important to use cv2.CHAIN_APPROX_SIMPLE to only retrieve the corner points
# corner points will be useful to construct transformation matrix in affine transform
contours, __ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours :
    # get bounding rect
    x, y, w, h = cv2.boundingRect(cnt)

    # filter out small rects
    if w < 50 or h < 50 :
        continue
    
    # find minimum enclosing object in clockwise direction
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = box.astype(int)

    
    # define four vertices of source & destination image
    # [top-left, top-right, bottom-right, bottom-left]
    pts1 = np.float32([box[1], box[0], box[3], box[2]]) # the corder of sudoku image
    pts2 = np.float32([[0, 0], [0,h], [w, h], [w, 0]]) # target image corners


    # create Perpective Transform Matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)

    # Apply Warp perspective transform
    output = cv2.warpPerspective(img, M, (w,h))



# display result
cv2.imshow("Original Image", img)
cv2.imshow("Perspective Transform Image", output)
cv2.waitKey(0)
cv2.destroyAllWindows()